In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader import *
import eicu_year_process

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [2]:
mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis.csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis.csv.gz'

mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test)

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type)

========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: 1.0    473941
2.0      5128
3.0     11145
4.0     19613
Name: Case, dtype: int64
테스트셋 클래스 비율: 1.0    53103
2.0      512
3.0      873
4.0     1590
Name: Case, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.93:0.01:0.02:0.04
테스트셋 클래스 비율: 0.95:0.01:0.02:0.03
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 2907
테스트셋 환자 수: 329
학습셋 체류 수: 2961
테스트셋 체류 수: 330
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.05
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.38
시도한 시행 횟수: 0


In [3]:
X_train, y_train, _ = split.split_X_Y(mimic_train, mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid, mode = 'mimic')

In [4]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X_num_scaled = scaler.fit_transform(X_train_case1_2)
# X_train_case1_2_scaled = pd.DataFrame(X_num_scaled,columns = X_train_case1_2.columns)

# X_valid_case1_2_scaled = scaler.transform(X_valid_case1_2)
# X_valid_case1_2_scaled = pd.DataFrame(X_valid_case1_2_scaled,columns = X_valid_case1_2.columns)

# X_test_case1_2_scaled = scaler.transform(X_test_case1_2)
# X_test_case1_2_scaled = pd.DataFrame(X_test_case1_2_scaled,columns = X_test_case1_2.columns)

In [21]:
reload(Multiclass_evaluation) 
models, result = Multiclass_evaluation.create_analysis(eventset, X_train, y_train, X_valid, valid_output)

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.979956,0.9239,0.9958,0.5601,0.3413,0.7677,0.6132,0.9265,0.8186,0.9991,0.5076,0.6015,0.8143
1,rf,0.982542,0.9191,0.9955,0.5438,0.4053,0.8356,0.7146,0.9320,0.8512,0.9995,0.4558,0.7101,0.8489
2,dt,0.955455,0.9181,0.9954,0.6019,0.1537,0.8886,0.7716,0.9300,0.8726,0.9680,0.4785,0.7698,0.8706
3,lr,0.948144,0.5312,0.9750,0.5000,0.5046,0.5037,0.1015,0.5259,0.2858,0.9788,0.0000,0.0154,0.0964


----------------------------------------------------------------------


In [43]:
reload(Multiclass_evaluation)
result = Multiclass_evaluation.external_evaluation(unitadmitsource, unittype, unitstaytype, models[1], 'RF', event_eicu)

[Starting eICU-Test]
eICU-UnitAdmitSource...
eICU-UnitType...
eICU-UnitStayType...


Model  Accuracy  Case 1 AUROC  \
Subpopulation   eICU Type                                            
UnitAdmitSource Floor                   RF  0.984409        0.9085   
                Acute Care/Floor        RF  0.986032        0.8752   
                Emergency Department    RF  0.988596        0.9164   
                Direct Admit            RF  0.993687        0.9129   
                Operating Room          RF  0.991540        0.9223   
                Step-Down Unit (SDU)    RF  0.986469        0.8854   
                Other ICU               RF  0.989440        0.8454   
                Recovery Room           RF  0.994145        0.9479   
                Other Hospital          RF  0.989971        0.9292   
                ICU                     RF  0.988436        0.8169   
                Chest Pain Center       RF  0.971069        0.8361   
                PACU                    RF  0.989965        0.9553   
UnitType        Med-Surg ICU            RF  0.988335        0.9135   
                MICU                    RF  0.988501        0.9106   
                Cardiac ICU             RF  0.988729        0.9061   
                Neuro ICU               RF  0.995185        0.9032   
                CTICU                   RF  0.991007        0.8891   
                CSICU                   RF  0.982067        0.9057   
                CCU-CTICU               RF  0.986918        0.8911   
                SICU                    RF  0.990496        0.9436   
UnitStayType    admit                   RF  0.989487        0.9130   
                transfer                RF  0.986789        0.9065   
                readmit                 RF  0.987588        0.9115   
                stepdown/other          RF  0.993882        0.4997   

                                      Case 1 AUPRC  Case 2 AUROC  \
Subpopulation   eICU Type                                          
UnitAdmitSource Floor                       0.9956        0.5363   
                Acute Care/Floor            0.9953        0.5215   
                Emergency Department        0.9965        0.5349   
                Direct Admit                0.9978        0.5294   
                Operating Room              0.9976        0.5197   
                Step-Down Unit (SDU)        0.9945        0.5206   
                Other ICU                   0.9963        0.5113   
                Recovery Room               0.9985        0.5286   
                Other Hospital              0.9977        0.5266   
                ICU                         0.9964        0.5247   
                Chest Pain Center           0.9894        0.4994   
                PACU                        0.9949        0.5342   
UnitType        Med-Surg ICU                0.9965        0.5352   
                MICU                        0.9965        0.5299   
                Cardiac ICU                 0.9960        0.5139   
                Neuro ICU                   0.9981        0.5166   
                CTICU                       0.9978        0.5202   
                CSICU                       0.9945        0.5326   
                CCU-CTICU                   0.9959        0.5177   
                SICU                        0.9975        0.5449   
UnitStayType    admit                       0.9968        0.5306   
                transfer                    0.9963        0.5229   
                readmit                     0.9964        0.5334   
                stepdown/other              0.9973        0.4997   

                                      Case 2 AUPRC Case 3 AUROC Case 3 AUPRC  \
Subpopulation   eICU Type                                                      
UnitAdmitSource Floor                       0.1906       0.7771       0.7045   
                Acute Care/Floor            0.0899       0.8265       0.7408   
                Emergency Department        0.1797       0.8488       0.7782   
                Direct Admit                0.1

In [44]:
result.to_excel(excel_writer='external_validation_baseline_method.xlsx')